In [1]:
# Import Libraries
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from functools import partial
import re
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import string

c:\Users\rakti\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Import Inventor Disambiguated CSV File
df = pd.read_csv('g_inventor_disambiguated.csv')

C:\Users\rakti\AppData\Local\Temp\ipykernel_48556\1022710257.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('g_inventor_disambiguated.csv')


In [3]:
# Concatenate the first and last names
df["full_name"] = df["disambig_inventor_name_first"].str.cat(df["disambig_inventor_name_last"], sep=" ")

In [4]:
# Clean the names by removing punctuation and converting to lowercase
df["cleaned_name"] = df["full_name"].str.replace(f"[{string.punctuation}]", "").str.lower()

C:\Users\rakti\AppData\Local\Temp\ipykernel_48556\908576034.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["cleaned_name"] = df["full_name"].str.replace(f"[{string.punctuation}]", "").str.lower()


In [5]:
# View DF head
df.head()

,Unnamed: 0,patent_id,inventor_sequence,inventor_id,disambig_inventor_name_first,disambig_inventor_name_last,male_flag,attribution_status,location_id,full_name,cleaned_name
0,0,10000000,0,fl:jo_ln:marron-5,Joseph,Marron,1.0,1,198b0471-16c8-11ed-9b5f-1234bde3cd05,Joseph Marron,joseph marron
1,1,10000001,1,fl:hy_ln:yu-30,Hyeon Jae,YU,NaN,99,3eb37495-16c8-11ed-9b5f-1234bde3cd05,Hyeon Jae YU,hyeon jae yu
2,2,10000001,0,fl:su_ln:lee-389,Sun-Woo,Lee,NaN,98,6c4ba08f-16c8-11ed-9b5f-1234bde3cd05,Sun-Woo Lee,sunwoo lee
3,3,10000002,2,fl:do_ln:choi-31,Dong-Hyeon,Choi,1.0,1,755a6338-16c8-11ed-9b5f-1234bde3cd05,Dong-Hyeon Choi,donghyeon choi
4,4,10000002,3,fl:do_ln:kim-369,Dong Jin,Kim,1.0,1,755a6338-16c8-11ed-9b5f-1234bde3cd05,Dong Jin Kim,dong jin kim


In [6]:
# Create a SPARQLWrapper object and set the endpoint URL
sparql = SPARQLWrapper("https://makg.org/sparql")

In [7]:
# Create Custom Function to loop
def get_data_for_inventor(specific_string):

    filtered_df = df[df['inventor_id'].str.contains(specific_string, case=False)]

    res = []
    first_name = filtered_df["disambig_inventor_name_first"].iloc[0]
    last_name = filtered_df["disambig_inventor_name_last"].iloc[0]
    

    for patent in filtered_df["patent_id"]:
        patent = str(patent)
        sparql.setQuery(f"""
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX magc: <https://makg.org/class/>
            PREFIX mag: <https://makg.org/property/>
            PREFIX dcterms: <http://purl.org/dc/terms/>
            PREFIX foaf: <http://xmlns.com/foaf/0.1/>
            PREFIX fabio: <http://purl.org/spar/fabio/>
            PREFIX prism: <http://prismstandard.org/namespaces/basic/2.0/>
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
            PREFIX dbo: <http://dbpedia.org/ontology/>

            SELECT (STR(?patentNumber) AS ?patent) (STR(?paperTitle) AS ?title) (STR(?authorName) AS ?author) ?arank
            WHERE {{
                ?paper fabio:hasPatentNumber "{patent}"^^xsd:string ;
                        dcterms:creator ?author ;
                        dcterms:title ?paperTitle .
                ?author foaf:name ?authorName .
                ?author mag:rank ?arank .
                FILTER (
                (CONTAINS(LCASE(?authorName), LCASE("{first_name}")) || CONTAINS(LCASE(?authorName), LCASE("{last_name}"))) ||
                (CONTAINS(LCASE(?authorName), LCASE("{last_name}")) || CONTAINS(LCASE(?authorName), LCASE("{first_name}")))
                )
                
                BIND("{patent}"^^xsd:string AS ?patentNumber)
            }}
        """)

        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for result in results["results"]["bindings"]:
            patent_id = result["patent"]["value"]
            paper_title = result["title"]["value"]
            patent_author = result["author"]["value"]
            author_rank = result["arank"]["value"]
            similarity_score = fuzz.ratio(patent_author.lower(), f"{first_name} {last_name}".lower())

            if similarity_score > 30:
                res2 = {
                    "Patent": patent_id,
                    "Paper_Title": paper_title,
                    "Patent_author": patent_author,
                    "Author_Rank": author_rank,
                    "Similarity_Score": similarity_score
                }

                res.append(res2)

    new_df = pd.DataFrame(res)

    res = []
    names = new_df["Patent_author"].unique()
    patents = set(new_df["Patent"])
    author_rank_set = set(new_df["Author_Rank"])

    def query_builder(first_name, last_name, name):
        
        sparql.setQuery(f"""
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX magc: <https://makg.org/class/>
            PREFIX dcterms: <http://purl.org/dc/terms/>
            PREFIX foaf: <http://xmlns.com/foaf/0.1/>
            PREFIX fabio: <http://purl.org/spar/fabio/>
            PREFIX prism: <http://prismstandard.org/namespaces/basic/2.0/>
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
            PREFIX mag: <https://makg.org/property/>
            PREFIX org: <http://www.w3.org/ns/org#>
            PREFIX dbo: <http://dbpedia.org/ontology/>
            
            SELECT (STR(?paperTitle) AS ?title) ?paperPubDate (STR(?patentNum) AS ?patentnumber) ?arank
                        (STR(?fname) AS ?fieldname) ?paperrank (STR(?affname) AS ?affname) (STR(?papjou) AS ?papjou)
                        ?conin (STR(?abstract) AS ?abstract)
            WHERE {{
                ?author foaf:name "{name}"^^xsd:string .
                ?paper dcterms:creator ?author .
                ?paper dcterms:title ?paperTitle .
                ?paper prism:publicationDate ?paperPubDate .
                ?author mag:rank ?arank .
                ?paper mag:rank ?paperrank .
                OPTIONAL {{ ?paper fabio:hasPatentNumber ?patentNum . }}
                OPTIONAL {{ ?paper fabio:hasDiscipline ?fieldofstudy .
                        ?fieldofstudy foaf:name ?fname . }}
                OPTIONAL {{ ?author org:memberOf ?affiliation . 
                        ?affiliation foaf:name ?affname . }}
                OPTIONAL {{ ?paper mag:appearsInJournal ?journal . 
                        ?journal foaf:name ?papjou . }}
                OPTIONAL {{ ?paper mag:citationCount ?conin . }}
                OPTIONAL {{ ?paper dcterms:abstract ?abstract . }}
                }}
        """)

        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        for result in results["results"]["bindings"]:
            title = result["title"]["value"]
            paper_pub_date = result["paperPubDate"]["value"]
            author_rank = result["arank"]["value"]
            paper_rank = result["paperrank"]["value"]
            field_of_study = result.get("fieldname", {}).get("value", "")
            aff_name = result.get("affname", {}).get("value", "")
            pap_jou = result.get("papjou", {}).get("value", "")
            con_in = result.get("conin", {}).get("value", None)
            abstract = result.get("abstract", {}).get("value", "")
            patent_num = result.get("patentnumber", {}).get("value", None)

            res2 = {
                "USPTO_ID": specific_string,
                "Common_Name_USPTO": f"{first_name} {last_name}",
                "Name_Variations_MAKG": name,
                "Similarity_Score": similarity_score,
                "Author_Affiliation": aff_name,
                "Author_Rank": author_rank,
                "Paper_Title": title,
                "Paper_Rank": paper_rank,
                "Paper_Field": field_of_study,
                "Paper_Journal": pap_jou,
                "Paper_Citation_Count": con_in,
                "Publication_Date": paper_pub_date,
                "Paper_Abstract": abstract
                
            }

            
            if patent_num:
                res2["Patent_Number"] = patent_num
                if patent_num in patents:
                    res.append(res2)
            else:
                res.append(res2)

    query_builder_partial = partial(query_builder, first_name, last_name)
    filtered_names = process.extractBests(f"{first_name} {last_name}", names, score_cutoff=30, scorer=fuzz.token_sort_ratio)

    for name in filtered_names:
        similarity_score = fuzz.ratio(name[0].lower(), f"{first_name} {last_name}".lower())
        if similarity_score > 30:
            query_builder_partial(name[0])

    pub_df = pd.DataFrame(res)

    # Remove duplicate entries for Paper_Rank where Patent_Number is not unique
    unique_patents = pub_df[pub_df['Patent_Number'].isna()].drop_duplicates(subset=['Paper_Rank'])
    duplicated_patents = pub_df[~pub_df['Patent_Number'].isna()].drop_duplicates(subset=['Patent_Number', 'Paper_Rank'])

    new_df = pd.concat([unique_patents, duplicated_patents])
    final_df = new_df.reset_index(drop=True)  # Reset the index

    return final_df


In [8]:
# Get the first 15 unique inventor IDs
unique_inventor_ids = df["inventor_id"].unique()[:15]

In [9]:
# Create an empty dataframe to store the results
result_df = pd.DataFrame()

# Loop through the unique inventor IDs
for inventor_id in unique_inventor_ids:
    inventor_data = get_data_for_inventor(inventor_id)
    result_df = pd.concat([result_df, inventor_data], ignore_index=True)

# Print the resulting dataframe
result_df

,USPTO_ID,Common_Name_USPTO,Name_Variations_MAKG,Similarity_Score,Author_Affiliation,Author_Rank,Paper_Title,Paper_Rank,Paper_Field,Paper_Journal,Paper_Citation_Count,Publication_Date,Paper_Abstract,Patent_Number
0,fl:jo_ln:marron-5,Joseph Marron,Joseph Marron,100,The Institute of Optics,18479,"Image-plane speckle from rotating, rough objects",22615,Physics,Journal of The Optical Society of America A-op...,11,1985-09-01,Time-varying laser speckle in the image plane ...,NaN
1,fl:jo_ln:marron-5,Joseph Marron,Joseph Marron,100,The Institute of Optics,18479,Accuracy of Fourier-magnitude estimation from ...,22798,Physics,Journal of The Optical Society of America A-op...,7,1988-06-01,The accuracy of Fourier-magnitude estimates ma...,NaN
2,fl:jo_ln:marron-5,Joseph Marron,Joseph Marron,100,The Institute of Optics,18479,Correlation properties of clipped laser speckle,22201,Physics,Journal of The Optical Society of America A-op...,22,1985-09-01,The technique of signal clipping is applied to...,NaN
3,fl:jo_ln:marron-5,Joseph Marron,Joseph Marron,100,The Institute of Optics,18479,Image recognition in the presence of laser spe...,23340,Engineering,Journal of The Optical Society of America A-op...,7,1986-07-01,Two techniques for performing digital image re...,NaN
4,fl:jo_ln:marron-5,Joseph Marron,Joseph Marron,100,,21075,Special Section Guest Editorial: Computational...,22909,Engineering,Optical Engineering,0,2015-03-01,This is the guest editorial for the Optical En...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2404,fl:ma_ln:saelen-1,Marc Saelen,Marc Saelen,100,,21075,Dreischneider Three Knife Trimmer,25522,,,0,2001-03-22,,NaN
2405,fl:ma_ln:saelen-1,Marc Saelen,Saelen Marc,55,,18770,Thermoforming mold device and a process for it...,21604,Materials science,,2,2015-12-17,A thermoforming mold device (1) providing a pi...,10000006
2406,fl:ma_ln:saelen-1,Marc Saelen,Saelen Marc,55,,18770,Vacuum thermoforming mold device and a manufac...,21989,Engineering,,1,2017-06-15,"A molding device for thermoforming, comprising...",10343329
2407,fl:ma_ln:saelen-1,Marc Saelen,Marc Saelen,100,,21075,Selective cutting one shot,21638,Engineering,,8,2001-03-22,The invention relates to a process and a devic...,7152296


In [10]:
# To Export as a CSV
# result_df.to_csv("First_15_inventors_without_AuthorRank_Consideration.csv",encoding="utf-8")